In [112]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import sys
import csv
import json
from io import StringIO
from lxml import objectify
import requests
import sqlite3

In [12]:
df = pd.read_csv('ex1.csv')
df
pd.read_table('ex1.csv', sep = ',')
pd.read_csv('ex2.csv', header = None)
pd.read_csv('ex2.csv', names = ['a', 'b', 'c', 'd', 'messages'])
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('ex2.csv', index_col = 1)

parsed = pd.read_csv('csv_mindex.csv', index_col = ['key1', 'key2'])
parsed
list(open('ex3.txt'))
result = pd.read_table('ex3.txt', sep = '\s+')
result
list(open('ex4.csv'))
pd.read_csv('ex4.csv', skiprows = [0, 2, 3])
result = pd.read_csv('ex5.csv')
result
pd.isnull(result)
result = pd.read_csv('ex5.csv', na_values = ['NULL'])
result
sentinels = {'message': ['foo'], 'something': ['two', 'three']}
pd.read_csv('ex5.csv', na_values = sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,NaN,9,10,11.0,12,NaN


In [68]:
result = pd.read_csv('ex6.csv')
result
pd.read_csv('ex6.csv', nrows = 5)
chunker = pd.read_csv('ex6.csv', chunksize = 1000)
chunker
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value = 0)
#   用到fill_alue，保证加法，否则就会导致结果全是NAN
tot = tot.sort_values(ascending = False)
tot[: 10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [13]:
data = pd.read_csv('ex5.csv')
data
data.to_csv('out.csv')
#data.to_csv(sys.stdout, sep = '|')
#data.to_csv(sys.stdout, na_rep = 'NULL')
#data.to_csv(sys.stdout, index = False, header = False)
#data.to_csv(sys.stdout, index = False, columns = ['a', 'b', 'c'])
dates = pd.date_range('1/1/2000', periods = 7)
dates
ts = Series(np.arange(7), index = dates)
ts.to_csv('tseries.csv')
#with open('tseries.csv') as f:
#    print(f.read())

Series.from_csv('tseries.csv', parse_dates = True)

/home/haoran/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:3727: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

f = open()
reader = csv.reader(f)
for line in reader:
    print(line)
d = pd.read_table('ex7.csv')
d

In [18]:
f = open('ex7.csv')
reader = csv.reader(f)
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [49]:
lines = list(csv.reader(open('ex7.csv')))
header, values = lines[0], lines[1: ]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL #   不可缺省
#reader = csv.reader(f, dialect = my_dialect)
#reader = csv.reader(f, delimiter = '|')
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect = my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))


In [73]:
tag = '<a href="http://www.goole.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()
root
root.get('href')
root.text


'Google'

In [80]:
frame = pd.read_csv('ex1.csv')
frame
frame.to_pickle('frame_pickle')#   save改成了to_pickle，load改成了read_pickle
pd.read_pickle('frame_pickle')


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [96]:
store = pd.HDFStore('mydata.h5')
store
store['obj1'] = frame
store['obj1_col'] = frame['a']
store['obj1_col']
xls_file = pd.ExcelFile('ex1.xlsx')
table = xls_file.parse('Sheet1')
table

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [111]:
url = 'https://s.weibo.com/weibo/python%2520pandas?topnav=1&wvr=6&b=1#_loginLayer_1597202766100'
resp = requests.get(url)
resp
t = DataFrame(np.arange(4).reshape((2, 2)), columns = ['a', 'b'], index = ['e', 'r'])
t

,a,b
e,0,1
r,2,3


In [134]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL,         d INTEGER
);"""
con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()
data = [('Atlanta', 'Georgia', 1.25, 6),
       ('Tallshassee', 'Florida', 2.6, 3),
       ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows
cursor.description
DataFrame(rows, columns = list(zip(*cursor.description))[0])#   list包装zip
import pandas.io.sql as sql
sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallshassee,Florida,2.60,3
2,Sacramento,California,1.70,5
